In [ ]:
#Importing all the required libraries
import pandas as pd
import numpy as np
import xgboost as xg
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#reading the provided csv files
df_train_features = pd.read_csv("/content/training_set_features.csv")
df_train_labels = pd.read_csv("/content/training_set_labels.csv")
df_test = pd.read_csv("/content/test_set_features.csv")

**DATA CLEANING**

In [ ]:
#Checking for the shape of the given files
df_train_features.shape, df_train_labels.shape

((26707, 36), (26707, 3))

In [ ]:
#Checking for the null values
df_train_features.isnull().sum()

respondent_id                      0
h1n1_concern                      92
h1n1_knowledge                   116
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_h1n1                2160
doctor_recc_seasonal            2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_h1n1_vacc_effective      391
opinion_h1n1_risk                388
opinion_h1n1_sick_from_vacc      395
opinion_seas_vacc_effective      462
opinion_seas_risk                514
opinion_seas_sick_from_vacc      537
age_group                          0
education                       1407
race                               0
sex                                0
income_poverty                  4423
m

In [ ]:
df_train_labels.isnull().sum()

respondent_id       0
h1n1_vaccine        0
seasonal_vaccine    0
dtype: int64

In [ ]:
#As we have a lot of null values in some of these columns we are droping it
df_train_features.drop(['employment_industry', 'employment_occupation'], axis=1, inplace=True)
df_test.drop([ 'employment_industry', 'employment_occupation'], axis=1, inplace=True)
df_train_features.columns

Index(['respondent_id', 'h1n1_concern', 'h1n1_knowledge',
       'behavioral_antiviral_meds', 'behavioral_avoidance',
       'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group',
       'education', 'race', 'sex', 'income_poverty', 'marital_status',
       'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa',
       'household_adults', 'household_children'],
      dtype='object')

In [ ]:
#Looking for data types of the columns
df_train_features.dtypes

respondent_id                    int64
h1n1_concern                   float64
h1n1_knowledge                 float64
behavioral_antiviral_meds      float64
behavioral_avoidance           float64
behavioral_face_mask           float64
behavioral_wash_hands          float64
behavioral_large_gatherings    float64
behavioral_outside_home        float64
behavioral_touch_face          float64
doctor_recc_h1n1               float64
doctor_recc_seasonal           float64
chronic_med_condition          float64
child_under_6_months           float64
health_worker                  float64
health_insurance               float64
opinion_h1n1_vacc_effective    float64
opinion_h1n1_risk              float64
opinion_h1n1_sick_from_vacc    float64
opinion_seas_vacc_effective    float64
opinion_seas_risk              float64
opinion_seas_sick_from_vacc    float64
age_group                       object
education                       object
race                            object
sex                      

In [ ]:
#Defining a common functions for cleaning data
def Manipulator(col,im):
  for i in col:
    df_train_features[i] = im.fit_transform(df_train_features[[i]])
    df_test[i] = im.transform(df_test[[i]])

In [ ]:
#Now checking for categorical values
col1 = []
for i in df_train_features.columns:
  if df_train_features[i].dtype == object:
    col1.append(i)

In [ ]:
#Changing categorical values to numerical values
label_encoder = preprocessing.LabelEncoder()
Manipulator(col1,label_encoder)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change th

In [ ]:
#Making a list of all null columns
col = []
for i in df_train_features.columns:
  if df_train_features[i].hasnans == True:
    col.append(i)

In [ ]:
#Using Simple Impputer to fill those null values
im = SimpleImputer(strategy = 'mean')
Manipulator(col,im)

**RESULTS AND MODEL PREDICTIONS**

In [ ]:
#Performing Train Test Split to check accuracies
X_train,X_val,y_train,y_val = train_test_split(df_train_features,df_train_labels['h1n1_vaccine'])
#Trying out XGboost
model1 = xg.XGBRegressor(n_estimators = 100,seed = 100,random_state = 0)

In [ ]:
model1.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=0, ...)

In [ ]:
#using Randomized Search for better predictions
params = {
    "n_estimators" : [50, 100, 150, 200],
 "learning_rate" : [0.05,0.10,0.15,0.20,0.25,0.30],
 "max_depth" : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma": [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
}

rs_model=RandomizedSearchCV(model1,param_distributions=params,n_iter=25,scoring='roc_auc',n_jobs=-1,cv=10,verbose=3)
rs_model.fit(X_train,y_train)
print(rs_model.best_estimator_)
print(rs_model.best_score_)

Fitting 10 folds for each of 25 candidates, totalling 250 fits
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=7, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=0, ...)
0.8670721425580108


In [ ]:
#Putting first prediction into a dataframe
Submission_df = pd.DataFrame()

In [ ]:
Submission_df

""


In [ ]:
Submission_df['respondent_id'] = df_test['respondent_id']

In [ ]:
Submission_df.set_index('respondent_id',inplace = True)

In [ ]:
predict_h1n1 = rs_model.predict(df_test)

In [ ]:
Submission_df['h1n1_vaccine'] = abs(predict_h1n1.round(1))

In [ ]:
Submission_df

,h1n1_vaccine
respondent_id,
26707,0.2
26708,0.0
26709,0.3
26710,0.5
26711,0.4
...,...
53410,0.3
53411,0.2
53412,0.2


In [ ]:
#Performing Similar steps for seasonal vaccine
X_train,X_val,y_train,y_val = train_test_split(df_train_features,df_train_labels['seasonal_vaccine'])
model1 = xg.XGBRegressor(n_estimators = 100,seed = 100,random_state = 0)

In [ ]:
model1.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=0, ...)

In [ ]:
params = {
    "n_estimators" : [50, 100, 150, 200],
 "learning_rate" : [0.05,0.10,0.15,0.20,0.25,0.30],
 "max_depth" : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma": [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
}

rs_model=RandomizedSearchCV(model1,param_distributions=params,n_iter=25,scoring='roc_auc',n_jobs=-1,cv=10,verbose=3)
rs_model.fit(X_train,y_train)
print(rs_model.best_estimator_)
print(rs_model.best_score_)

Fitting 10 folds for each of 25 candidates, totalling 250 fits
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=7, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=150, n_jobs=None,
             num_parallel_tree=None, random_state=0, ...)
0.8607029438332748


In [ ]:
predict_seasonal = rs_model.predict(df_test)

In [ ]:
Submission_df['seasonal_vaccine'] = abs(predict_seasonal.round(1))

In [ ]:
Submission_df

,h1n1_vaccine,seasonal_vaccine
respondent_id,,
26707,0.2,0.2
26708,0.0,0.1
26709,0.3,0.7
26710,0.5,0.8
26711,0.4,0.4
...,...,...
53410,0.3,0.5
53411,0.2,0.3
53412,0.2,0.3


In [ ]:
Submission_df.to_csv('Submission_csv_final')